In [1]:
import json

In [2]:
import pandas as pd
from pandas.io.json import json_normalize

In [3]:
import numpy as np
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
import pickle
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kaurl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
df = pd.read_json('News_Category_Dataset_v2.json', lines=True)

In [5]:
df

,authors,category,date,headline,link,short_description
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ..."
5,Ron Dicker,ENTERTAINMENT,2018-05-26,Morgan Freeman 'Devastated' That Sexual Harass...,https://www.huffingtonpost.com/entry/morgan-fr...,"""It is not right to equate horrific incidents ..."
6,Ron Dicker,ENTERTAINMENT,2018-05-26,Donald Trump Is Lovin' New McDonald's Jingle I...,https://www.huffingtonpost.com/entry/donald-tr...,"It's catchy, all right."
7,Todd Van Luling,ENTERTAINMENT,2018-05-26,What To Watch On Amazon Prime That’s New This ...,https://www.huffingtonpost.com/entry/amazon-pr...,There's a great mini-series joining this week.
8,Andy McDonald,ENTERTAINMENT,2018-05-26,Mike Myers Reveals He'd 'Like To' Do A Fourth ...,https://www.huffingtonpost.com/entry/mike-myer...,"Myer's kids may be pushing for a new ""Powers"" ..."
9,Todd Van Luling,ENTERTAINMENT,2018-05-26,What To Watch On Hulu That’s New This Week,https://www.huffingtonpost.com/entry/hulu-what...,You're getting a recent Academy Award-winning ...


In [6]:
df.drop(['authors', 'link'], axis = 1, inplace = True)

In [7]:
train_size = int(len(df) * .8)
train_posts = df['headline'][:train_size]
train_tags = df['category'][:train_size]
test_posts = df['headline'][train_size:]
test_tags = df['category'][train_size:]

In [8]:
import keras

Using TensorFlow backend.


In [9]:
vocab_size = 1000
tokenize = keras.preprocessing.text.Tokenizer(num_words=vocab_size, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ', char_level=False, oov_token=None, document_count=0)
tokenize.fit_on_texts(train_posts)

In [10]:
x_train = tokenize.texts_to_matrix(train_posts)

In [11]:
x_test = tokenize.texts_to_matrix(test_posts)

In [12]:
x_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [13]:
from sklearn import preprocessing

In [14]:
encoder = preprocessing.LabelBinarizer()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

In [15]:
model = keras.models.Sequential()

In [16]:
from keras.layers import Activation, Dense

In [17]:
model.add(Dense(93, input_shape=(vocab_size,)))
model.add(Activation('sigmoid'))

Instructions for updating:
Colocations handled automatically by placer.


In [18]:
num_labels = 41

In [19]:
model.add(Dense(num_labels))
model.add(Activation('relu'))

In [20]:
model.compile(loss='binary_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy'])

In [21]:
batch_size = 16

In [22]:
history = model.fit(x_train, y_train, 
                    batch_size=batch_size, 
                    epochs=3, 
                    verbose=1, 
                    validation_split=0.1)

Instructions for updating:
Use tf.cast instead.
Train on 144613 samples, validate on 16069 samples
Epoch 1/3
144613/144613 [==============================] - 22s 151us/step - loss: 0.4814 - accuracy: 0.9643 - val_loss: 0.3866 - val_accuracy: 0.9737
Epoch 2/3
144613/144613 [==============================] - 19s 132us/step - loss: 0.2953 - accuracy: 0.9768 - val_loss: 0.3695 - val_accuracy: 0.9742
Epoch 3/3
144613/144613 [==============================] - 20s 142us/step - loss: 0.2900 - accuracy: 0.9770 - val_loss: 0.3760 - val_accuracy: 0.9748


In [23]:
score = model.evaluate(x_test, y_test, 
                       batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

40171/40171 [==============================] - 2s 62us/step
Test score: 0.3816347716777314
Test accuracy: 0.9745432138442993


In [28]:
print(model.metrics_names)

['loss', 'accuracy']


In [24]:
for i in range(10):    
    prediction = model.predict(np.array([x_test[i]]))

In [35]:
text_labels = encoder.classes_ 
predicted_label = text_labels[np.argmax(prediction[0])]
print(test_posts.iloc[i][:50], "...")
print('Actual label:' + test_tags.iloc[i])
print("Predicted label: " + predicted_label)

Shots Of Anesthetic Kill Budding Wisdom Teeth, Stu ...
Actual label:WELLNESS
Predicted label: POLITICS


In [26]:
import StringClassification

In [27]:
StringClassification.main()

KeyboardInterrupt: 